Chạy đoạn code này trên Kaggle/Colab để train

In [ ]:
# 1. Cài ultralytics và roboflow (phiên bản mới nhất)
#!pip install --quiet ultralytics roboflow

# 2. Kiểm tra phiên bản các thư viện chính
import ultralytics
import roboflow

print("Ultralytics YOLOv8 version:", ultralytics.__version__)
print("Roboflow SDK version:", roboflow.__version__)

# 3. Test import các thư viện thường dùng (OpenCV, torch)
import cv2
import torch

print("OpenCV version:", cv2.__version__)
print("PyTorch version:", torch.__version__)

Huấn luyện model, data lấy từ roboflow (được được chia tập train - test - val)

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import os

rf = Roboflow(api_key="STe901OMGDcnmRX0vKI3")
project = rf.workspace("carplatedetection-8plix").project("carplatedetection-qv4tz")
version = project.version(9)
dataset = version.download("yolov8")

model = YOLO('yolov8n.pt')
data_yaml_path = os.path.join(dataset.location, "data.yaml")

# Các siêu tham số: epochs: 30, imgsz: 640 px, batch: 16

model.train(
    data=data_yaml_path,
    epochs=30,
    imgsz=640,
    batch=16,
    name='yolov8n-carplate',
    project='runs/train'
)

In ra các chỉ số cần thiết để đánh giá

In [ ]:
metrics = model.val()

results = metrics.results_dict

# Precision và Recall
precision = results['metrics/precision(B)']
recall = results['metrics/recall(B)']

# Tính F1-score
f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)  # tránh chia 0

# In các chỉ số đánh giá
print(f"📊 Precision     : {precision:.4f}")
print(f"📊 Recall        : {recall:.4f}")
print(f"📊 F1-score      : {f1_score:.4f}")
print(f"📊 mAP@0.5       : {results['metrics/mAP50(B)']:.4f}")
print(f"📊 mAP@0.5:0.95  : {results['metrics/mAP50-95(B)']:.4f}")

# In ma trận nhầm lẫn
print("\n📉 Confusion Matrix:")
print(metrics.confusion_matrix.matrix)